In [ ]:
import tensorflow as tf
import keras

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint 
from datetime import datetime 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, Activation, MaxPooling2D, GRU, Reshape, BatchNormalization

# Data Preproecssing

In [ ]:
#Fetch csv files
test_dir = 'test_image'
train_dir = 'train_image'

train_csv = "csv/train_data.csv"
test_csv = "csv/test_data.csv"

train = pd.DataFrame(data=pd.read_csv(train_csv,dtype = str, error_bad_lines=False))
test = pd.DataFrame(data=pd.read_csv(test_csv,dtype = str, error_bad_lines=False))

#Drop composers out of list
List = ['Ludwig van Beethoven', 'Wolfgang Amadeus Mozart', 'Johann Sebastian Bach', 'Franz Schubert', 'Frédéric Chopin']
train = train[train['composer'].isin(List)]
test = test[test['composer'].isin(List)]

num_labels = 5

In [ ]:
#Create datagenerator
traingen=ImageDataGenerator(rescale=1./255, validation_split = 0.1)
testgen=ImageDataGenerator(rescale=1./255)

train_generator=traingen.flow_from_dataframe(
      dataframe=train,
      directory = "train_image/",
      x_col="image_file",
      y_col="composer",
      subset="training",
      batch_size=32,
      seed=42,
      shuffle=True,
      class_mode="categorical",
      target_size=(32,96))

validation_generator = traingen.flow_from_dataframe(
    dataframe=train,
    directory = "train_image/",
    x_col="image_file",
    y_col="composer",
    subset="validation",
    target_size=(32,96),
    batch_size=32,
    class_mode='categorical')

test_generator=testgen.flow_from_dataframe(
  dataframe=test,
  directory = "test_image/",
  x_col="audio_file",
  y_col="composer",
  batch_size=32,
  seed=42,
  shuffle=True,
  class_mode="categorical",
  target_size=(32,96))

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = test_generator.n//test_generator.batch_size

# Single Vector Machine

In [ ]:
# One hot encoder for SVM
from sklearn.preprocessing import OneHotEncoder 
onehotencoder = OneHotEncoder(handle_unknown='ignore') 
enc_df = pd.DataFrame(onehotencoder.fit_transform(train[['composer']]).toarray())
train = train.join(enc_df)
onehotencoder = OneHotEncoder(handle_unknown='ignore') 
enc_df = pd.DataFrame(onehotencoder.fit_transform(test[['composer']]).toarray())
test = test.join(enc_df)

In [ ]:
#train_image
folder = Path("train_image/")
dirs = folder.glob("*")
labels_dict = {'Johann Sebastian Bach': 3 , 'Frédéric Chopin': 2, 'Wolfgang Amadeus Mozart': 4, 
               'Ludwig van Beethoven': 0, 'Franz Schubert': 1}

image_data = []
labels = [] 

for index, row in train.iterrows():
    img_path = Path(path+"train_image2/"+row["image_file"])
    img = image.load_img(img_path, target_size = (32,96))
    img_array = image.img_to_array(img)
    image_data.append(img_array)
    label = row[[2,3,4,5,6]].to_numpy()
    labels.append(label)

## Convert data into numpy array

train_data = np.array(image_data, dtype='float32')/255.0
labels = np.array(labels)
train_data = train_data.reshape(11633, 32*96*3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data, labels, test_size=0.1, random_state=42)
print(X_train.shape, X_test.shape)

In [ ]:
#Train SVM
from sklearn.svm import LinearSVC

clf = LinearSVC()
y_train = np.argmax(y_train, axis = 1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)
clf.score(X_train, y_train)

In [ ]:
#Predict SVM
y_test = np.argmax(y_test, axis = 1)
clf.score(X_test, y_test)

# Convolutional Neural Networks

In [ ]:
#CNN structure : 4-2dlayers & 2 connected layers
def buildCNN(num_labels, layers, weight=None):
    model = Sequential()
    
    channel_axis = 3
    
    model.add(Conv2D(layers[0], (3, 3), strides=(1, 1), input_shape = (32,96,3), padding = "same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(layers[1], (3, 3), padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(layers[2], (3, 3), padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.1))
    
    model.add(Conv2D(layers[3], (3, 3), padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.1))
        
    model.add(Flatten())
    model.add(Dropout(rate=0.5))

    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.5))

    model.add(Dense(num_labels))
    model.add(Activation('sigmoid'))

    if weight is None:
        return model

    else:
        model.load_weights(weight)
        return model

In [ ]:
#CNN Model
layers = [32, 64, 128, 128]
modelCNN = buildCNN(num_labels, layers)
modelCNN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelCNN.summary()

In [ ]:
#Train CNN
start = datetime.now()

num_epochs = 150

cp1 = ModelCheckpoint(filepath='saved_models/weights.best.basic_cnn_test.hdf5', verbose=1, save_best_only=True)
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)
historyCNN = modelCNN.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=validation_generator, 
                              validation_steps = STEP_SIZE_VALID, callbacks = [cp1, es], epochs=num_epochs)

duration = datetime.now() - start
print("Training completed in time: ", duration)
model.save('saved_models/cnn.h5')

In [ ]:
#Evaluate model
print(modelCNN.evaluate_generator(train_generator))
print(modelCNN.evaluate_generator(validation_generator))
print(modelCNN.evaluate_generator(test_generator))

# Convolutional Recurrent Neural Networks

In [ ]:
#CRNN model : 3-2d convolutional layers with 2 RNN layers
def buildCRNN(num_labels, weight=None):
    model = Sequential()
    
    model.add(Conv2D(64, (3, 3), strides=(1, 1), input_shape = (32,96,3), padding = "same"))
    model.add(MaxPooling2D((2, 2)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.1))

    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(MaxPooling2D((2, 2)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.1))

    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(MaxPooling2D((3, 3)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.1))
    
    model.add(Flatten())
    model.add(Dropout(rate=0.5))

    model.add(Reshape((16,128)))
    
    model.add(GRU(32, return_sequences=True))
    model.add(GRU(32, return_sequences=False))
    model.add(Dropout(rate=0.3))
    
    model.add(Dense(num_labels))
    model.add(Activation('sigmoid'))
    
    if weight is None:
        return model

    else:
        model.load_weights(weight)
        return model


In [ ]:
#Build CRNN
modelCRNN = buildCRNN(num_labels)
modelCRNN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelCRNN.summary()

In [ ]:
#Train CRNN
start = datetime.now()
num_epochs = 150

checkpointer2 = ModelCheckpoint(filepath='saved_models/weights.best.basic_crnn_test.hdf5', verbose=1, save_best_only=True)
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)
historyCRNN = modelCRNN.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=validation_generator, 
                                validation_steps=STEP_SIZE_VALID, callbacks = [checkpointer2, es], epochs=num_epochs)
duration = datetime.now() - start
print("Training completed in time: ", duration)
model2.save('saved_models/crnn.h5')

In [ ]:
#Evaluate model
print(modelCRNN.evaluate_generator(train_generator))
print(modelCRNN.evaluate_generator(validation_generator))
print(modelCRNN.evaluate_generator(test_generator))

# Confusion Matrix (Testing CRNN)

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix

Y_pred = modelCRNN.predict(train_generator)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(y_pred, train_generator.classes)

classes = ['Franz Schubert', 'Frédéric Chopin', 'Johann Sebastian Bach', 'Ludwig van Beethoven', 'Wolfgang Amadeus Mozart']

# Plot confusion matrix
plt.imshow(cm,interpolation='none',cmap='Blues')
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
for (i, j), z in np.ndenumerate(cm):
       plt.text(j, i, z, ha='center', va='center')
plt.show()